[View in Colaboratory](https://colab.research.google.com/github/sungreong/TIL/blob/master/Keras_Model_save_load_and_retrain.ipynb)

In [1]:
from __future__ import print_function
import keras
import os.path
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
# Path to saved model weights(as hdf5)
resume_weights = "./mnist-cnn-best.hdf5"

# Where to save Checkpoint(In the /output folder)
filepath = "./mnist-cnn-best.hdf5"

# Hyper-parameters
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [3]:
# MNIST handwritten image classification
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape strategy according to backend
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    # 1 x 28 x 28 [number_of_channels (colors) x height x weight]
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    # 28 x 28 x 1 [height x weight x number_of_channels (colors)]
    input_shape = (img_rows, img_cols, 1)

# Reshape, type, normalized, print
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Dataset info
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 3s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                    activation='relu',
                    input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [13]:
# If exists a best model, load its weights!
if os.path.isfile(resume_weights):
    print ("Resumed model's weights from {}".format(resume_weights))
    # load weights
    model.load_weights(resume_weights)

Resumed model's weights from ./mnist-cnn-best.hdf5


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])

In [0]:
checkpoint = ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

In [16]:
# Train
model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test, y_test),
                callbacks=[checkpoint])

# Eval
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 162s 3ms/step - loss: 0.0167 - acc: 0.9944 - val_loss: 0.0289 - val_acc: 0.9922

Epoch 00001: val_acc improved from -inf to 0.99220, saving model to ./mnist-cnn-best.hdf5
Epoch 2/12
10624/60000 [====>.........................] - ETA: 2:10 - loss: 0.0097 - acc: 0.9965

60000/60000 [==============================] - 164s 3ms/step - loss: 0.0136 - acc: 0.9955 - val_loss: 0.0297 - val_acc: 0.9926

Epoch 00002: val_acc improved from 0.99220 to 0.99260, saving model to ./mnist-cnn-best.hdf5
Epoch 3/12
27008/60000 [============>.................] - ETA: 1:26 - loss: 0.0120 - acc: 0.9954

60000/60000 [==============================] - 163s 3ms/step - loss: 0.0130 - acc: 0.9955 - val_loss: 0.0339 - val_acc: 0.9918

Epoch 00003: val_acc did not improve from 0.99260
Epoch 4/12
36352/60000 [=================>............] - ETA: 1:01 - loss: 0.0143 - acc: 0.9952

60000/60000 [==============================] - 162s 3ms/step - loss: 0.0148 - acc: 0.9950 - val_loss: 0.0275 - val_acc: 0.9929

Epoch 00004: val_acc improved from 0.99260 to 0.99290, saving model to ./mnist-cnn-best.hdf5
Epoch 5/12
33920/60000 [===============>..............] - ETA: 1:08 - loss: 0.0102 - acc: 0.9967

60000/60000 [==============================] - 162s 3ms/step - loss: 0.0109 - acc: 0.9964 - val_loss: 0.0337 - val_acc: 0.9923

Epoch 00005: val_acc did not improve from 0.99290
Epoch 6/12
38528/60000 [==================>...........] - ETA: 55s - loss: 0.0126 - acc: 0.9956

...

60000/60000 [==============================] - 161s 3ms/step - loss: 0.0121 - acc: 0.9959 - val_loss: 0.0310 - val_acc: 0.9934

Epoch 00008: val_acc improved from 0.99300 to 0.99340, saving model to ./mnist-cnn-best.hdf5
Epoch 9/12
34560/60000 [================>.............] - ETA: 1:05 - loss: 0.0117 - acc: 0.9957

60000/60000 [==============================] - 161s 3ms/step - loss: 0.0111 - acc: 0.9962 - val_loss: 0.0322 - val_acc: 0.9930

Epoch 00009: val_acc did not improve from 0.99340
Epoch 10/12
38528/60000 [==================>...........] - ETA: 55s - loss: 0.0105 - acc: 0.9962

60000/60000 [==============================] - 161s 3ms/step - loss: 0.0110 - acc: 0.9962 - val_loss: 0.0333 - val_acc: 0.9933

Epoch 00010: val_acc did not improve from 0.99340
Epoch 11/12
39808/60000 [==================>...........] - ETA: 52s - loss: 0.0101 - acc: 0.9965

60000/60000 [==============================] - 162s 3ms/step - loss: 0.0097 - acc: 0.9964 - val_loss: 0.0324 - val_acc: 0.9924

Epoch 00011: val_acc did not improve from 0.99340
Epoch 12/12
40192/60000 [===================>..........] - ETA: 51s - loss: 0.0097 - acc: 0.9966

60000/60000 [==============================] - 163s 3ms/step - loss: 0.0095 - acc: 0.9965 - val_loss: 0.0329 - val_acc: 0.9924

Epoch 00012: val_acc did not improve from 0.99340
Test loss: 0.03288496296733142
Test accuracy: 0.9924


In [17]:
!ls

datalab  mnist-cnn-best.hdf5
